<a href="https://colab.research.google.com/github/JapiKredi/Transformer_code/blob/main/transformer_code2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

# Define the Positional Encoding module
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# Define the Multi-Head Attention module
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads
        self.linear_layers = nn.ModuleList([nn.Linear(d_model, d_model) for _ in range(4)])
        self.output_linear = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attention = F.softmax(scores, dim=-1)
        return torch.matmul(attention, V)

    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)

        # Linear transformation for Query, Key, and Value
        Q, K, V = [l(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
                   for l, x in zip(self.linear_layers, (Q, K, V))]

        # Apply scaled dot-product attention
        scaled_attention = self.scaled_dot_product_attention(Q, K, V, mask)

        # Concatenate attention heads and apply final linear layer
        concatenated_attention = scaled_attention.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.d_k)
        return self.output_linear(concatenated_attention)

# Define the Feed Forward module
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(p=0.1)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x

# Define the Encoder Layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super(EncoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)

    def forward(self, x, mask):
        attention_output = self.self_attention(x, x, x, mask)
        attention_output = self.layer_norm1(x + attention_output)
        ff_output = self.feed_forward(attention_output)
        return self.layer_norm2(attention_output + ff_output)

# Define the Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, d_ff):
        super(TransformerEncoder, self).__init__()
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff) for _ in range(num_layers)])

    def forward(self, x, mask):
        for layer in self.encoder_layers:
            x = layer(x, mask)
        return x

# Define the Transformer model
class Transformer(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, d_ff, input_vocab_size, output_vocab_size):
        super(Transformer, self).__init__()
        self.encoder = TransformerEncoder(num_layers, d_model, num_heads, d_ff)
        self.linear = nn.Linear(d_model, output_vocab_size)

    def forward(self, src, src_mask):
        enc_output = self.encoder(src, src_mask)
        output = self.linear(enc_output)
        return output

# Create a Transformer model instance
d_model = 512
num_layers = 6
num_heads = 8
d_ff = 2048
input_vocab_size = 10000  # Example vocabulary size for input
output_vocab_size = 10000  # Example vocabulary size for output
transformer_model = Transformer(num_layers, d_model, num_heads, d_ff, input_vocab_size, output_vocab_size)

# Example usage
src = torch.tensor([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]])  # Example input sequence
src_mask = torch.ones_like(src).unsqueeze(1).unsqueeze(2)  # Example input mask
output = transformer_model(src, src_mask)
print(output.shape)  # Example output shape


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float